In [1]:
# Info on how to get your api key (kaggle.json) here: https://github.com/Kaggle/kaggle-api#api-credentials
# Valeria Quiroz  - Gabriel Zapata
%%time
!pip install kaggle
api_token = {"username":"gabrielzapata","key":"45d11139cb4e32995898eae373213b5f"}
import json
import zipfile
import os
with open('/content/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!mkdir /root/.kaggle      
!cp /content/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v/content
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c ga-customer-revenue-prediction
if not os.path.exists("/content/competitions/ga-customer-revenue-prediction"):
    os.makedirs("/content/competitions/ga-customer-revenue-prediction")
os.chdir('/content/competitions/ga-customer-revenue-prediction')
for file in os.listdir():
    zip_ref = zipfile.ZipFile(file, 'r')
    zip_ref.extractall()
    zip_ref.close()

- path is now set to: /content
 90% 55.0M/61.1M [00:00<00:00, 69.1MB/s]
100% 61.1M/61.1M [00:00<00:00, 78.5MB/s]
  0% 0.00/5.23M [00:00<?, ?B/s]
100% 5.23M/5.23M [00:00<00:00, 48.0MB/s]
  0% 0.00/2.55M [00:00<?, ?B/s]
100% 2.55M/2.55M [00:00<00:00, 84.3MB/s]
 96% 321M/333M [00:03<00:00, 82.8MB/s]
100% 333M/333M [00:03<00:00, 97.8MB/s]
100% 1.11G/1.11G [00:15<00:00, 29.0MB/s]
100% 1.11G/1.11G [00:15<00:00, 78.6MB/s]
 99% 56.0M/56.8M [00:00<00:00, 58.7MB/s]
100% 56.8M/56.8M [00:00<00:00, 65.0MB/s]
CPU times: user 1min 38s, sys: 33 s, total: 2min 11s
Wall time: 5min 13s


### **Librerias**

In [2]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from pandas import json_normalize
import seaborn as sns 
from sklearn import model_selection, preprocessing, metrics
import lightgbm as lgb
import plotly.graph_objs as go
from plotly import subplots
import plotly.offline as py
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
import datetime
from sklearn.model_selection import  GroupKFold
from sklearn.pipeline import Pipeline
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.layers import Dense, Dropout, SimpleRNN, RepeatVector, TimeDistributed, LSTM, GRU
import numpy as np
import tensorflow as tf,keras    
from tensorflow.keras.models import Sequential  
#from sklearn.metrics import mean_absolute_percentage_error

### **Explicacion del codigo json sacado de Kaggle, para aplanar los datos**

In [3]:
def load_df(csv_path='/content/competitions/ga-customer-revenue-prediction/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [4]:
%%time
train_df = load_df()
#test_df = load_df('/content/competitions/ga-customer-revenue-prediction/test.csv')

Loaded train.csv. Shape: (903653, 55)
CPU times: user 1min 58s, sys: 8.11 s, total: 2min 6s
Wall time: 2min 15s


In [5]:
train_df = train_df.drop( ['sessionId'] + ["trafficSource.campaignCode"], axis=1)
print(train_df.shape)

(903653, 53)


### **Eliminemos columnas constantes e imputar valores cero a NAN**

In [6]:
print ("Antes de eliminar columnas constantes - shape de train y test datasets: ", train_df.shape)
train_df = train_df.loc[:, (train_df != train_df.iloc[0]).any()]
print ("Despues de eliminar columnas constantes - shape de train y test datasets: ", train_df.shape)

Antes de eliminar columnas constantes - shape de train y test datasets:  (903653, 53)
Despues de eliminar columnas constantes - shape de train y test datasets:  (903653, 34)


In [7]:
# Replace NaN values throughout train dataset
train_df.replace(to_replace=np.nan, value=0, inplace=True)

### **Conversion variable categórica a númerica**

In [8]:
# etiqueta codifica las variables categóricas y convierte las variables numéricas flotantes
cat_cols = ["channelGrouping", "device.browser",
            "device.deviceCategory", "device.operatingSystem", "device.isMobile",
            "geoNetwork.city", "geoNetwork.continent", 
            "geoNetwork.country", "geoNetwork.metro",
            "geoNetwork.networkDomain", "geoNetwork.region", 
            "geoNetwork.subContinent", "trafficSource.adContent", 
            "trafficSource.adwordsClickInfo.adNetworkType", 
            "trafficSource.adwordsClickInfo.gclId", 
            "trafficSource.adwordsClickInfo.page", 
            "trafficSource.adwordsClickInfo.slot", "trafficSource.campaign",
            "trafficSource.keyword", "trafficSource.medium", 
            "trafficSource.referralPath", "trafficSource.source",
            'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect']
for col in cat_cols:
    print(col)
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train_df[col].values.astype('str')))
    train_df[col] = lbl.transform(list(train_df[col].values.astype('str')))
    


num_cols = ["totals.hits", "totals.pageviews", "visitNumber", "visitStartTime", 'totals.bounces',  'totals.newVisits']    
for col in num_cols:
    train_df[col] = train_df[col].astype(float)
   

channelGrouping
device.browser
device.deviceCategory
device.operatingSystem
device.isMobile
geoNetwork.city
geoNetwork.continent
geoNetwork.country
geoNetwork.metro
geoNetwork.networkDomain
geoNetwork.region
geoNetwork.subContinent
trafficSource.adContent
trafficSource.adwordsClickInfo.adNetworkType
trafficSource.adwordsClickInfo.gclId
trafficSource.adwordsClickInfo.page
trafficSource.adwordsClickInfo.slot
trafficSource.campaign
trafficSource.keyword
trafficSource.medium
trafficSource.referralPath
trafficSource.source
trafficSource.adwordsClickInfo.isVideoAd
trafficSource.isTrueDirect


In [9]:
train_df.to_csv('dataset.csv')

In [10]:
Y = np.log1p(train_df["totals.transactionRevenue"].astype('float'))
X=train_df.drop(["totals.transactionRevenue"],axis=1)##Eliminamos la salida de los datos de las caracteristicas
X=X.drop(["date"],axis=1)

In [11]:
X['fullVisitorId']=X['fullVisitorId'].values.astype('float')

In [12]:
groupsTrain = X['fullVisitorId']
group_kfold = GroupKFold(n_splits=2)

In [13]:
train_index, test_index = next(GroupKFold(n_splits=4).split(X,Y, groupsTrain)) 
X_train = X.iloc[train_index]
Y_train= Y.iloc[train_index]
X_test = X.iloc[test_index]
Y_test = Y.iloc[test_index]   
    

In [14]:
######Convesor a tensor######
def Conv2tensor(arg):
  arg = tf.convert_to_tensor(arg, dtype=tf.float64)
  return arg

In [15]:
X_traint = Conv2tensor(X_train)
Y_traint= Conv2tensor(Y_train)
X_testt = Conv2tensor(X_test)
Y_testt = Conv2tensor(Y_test)   

In [16]:
###Experimento 2
def model2(Neuronas=32 ,dropout=0.1 ,Acti='relu'):
    model = Sequential()
    model.add(Dense(Neuronas, input_dim=32, activation=Acti))
    model.add(Dropout(dropout))
    model.add(Dense(Neuronas, activation=Acti))
    model.add(Dropout(dropout))
    model.add(Dense(1,activation ='sigmoid'))
    model.compile(optimizer='Adam',loss='mse',metrics=[tf.keras.metrics.RootMeanSquaredError(),tf.keras.metrics.MeanAbsoluteError()])
    return model

In [17]:
groups = X_train['fullVisitorId']
group_kfold= GroupKFold(n_splits=3).split(X_train,Y_train, groups) 

In [18]:
# Grid de hiperparámetros evaluados
# ==============================================================================
num_cell=[64,128,256,512,1028,2048]
Drop=[0.1,0.3,0.5,0.7,0.9]
Act=['relu','tanh','sigmoid', 'linear']
param_grid = dict(Neuronas=num_cell,dropout=Drop,Acti=Act)
estimator=KerasRegressor(build_fn=model2,batch_size=800,epochs=20)

# Búsqueda por grid search con validación cruzada
# ==============================================================================
grid =  GridSearchCV(estimator=estimator ,
                     param_grid = param_grid,
                     scoring = 'neg_root_mean_squared_error',
                     n_jobs     =  20,
                     cv         = group_kfold, 
                     refit      = True,
                     verbose    = 1,
                     return_train_score = True
                     )


In [ ]:
grid.fit(X_train,Y_train, groups)

Fitting 3 folds for each of 120 candidates, totalling 360 fits


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:  1.5min
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.



In [ ]:
TrainValRMSE=grid.cv_results_

In [ ]:
TrainValRMSE

In [ ]:
grid.best_params_

In [ ]:
ErrorValRMSE=-1*(TrainValRMSE.get('split0_test_score')[65]+TrainValRMSE.get('split1_test_score')[65]+TrainValRMSE.get('split2_test_score')[65])/3
ErrorValRMSE

In [ ]:
ErrorTrainRMSE=-1*(TrainValRMSE.get('split0_train_score')[65]+TrainValRMSE.get('split1_train_score')[65]+TrainValRMSE.get('split2_train_score')[65])/3
ErrorTrainRMSE

In [ ]:
-1*grid.best_score_#confirma el calculo del error de validacion

In [ ]:
Model=grid.best_estimator_

In [ ]:
pred_test_y1= Model.predict(X_testt)

In [ ]:
pred_test_y1[pred_test_y1<0]=0
print(f"RMSE= {np.sqrt(metrics.mean_squared_error(Y_test,pred_test_y1))}")
print(f"MAE= {metrics.mean_absolute_error(Y_test,pred_test_y1)}")

In [ ]:
test_pred_df = pd.DataFrame({"fullVisitorId":X_test["fullVisitorId"].values})
test_pred_df["transactionRevenue"] = (Y_test.values)
test_pred_df["PredictedRevenue"] = (pred_test_y1)
test_pred_df = test_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
print('RMSE')
print(np.sqrt(metrics.mean_squared_error(test_pred_df["transactionRevenue"].values, test_pred_df["PredictedRevenue"].values)))
print('MAE')
print((metrics.mean_absolute_error((test_pred_df["transactionRevenue"].values ),(test_pred_df["PredictedRevenue"].values))))

Scoring MAE

In [ ]:
groups = X_train['fullVisitorId']
group_kfold= GroupKFold(n_splits=3).split(X_train,Y_train, groups) 

In [ ]:
# Grid de hiperparámetros evaluados
# ==============================================================================
num_cell=[64,128,256,512,1028,2048]
Drop=[0.1,0.3,0.5,0.7,0.9]
Act=['relu','tanh','sigmoid', 'linear']
param_grid = dict(Neuronas=num_cell,dropout=Drop,Acti=Act)
estimator=KerasRegressor(build_fn=model2,batch_size=50,epochs=50)

# Búsqueda por grid search con validación cruzada
# ==============================================================================
grid1 =  GridSearchCV(estimator=estimator ,
                     param_grid = param_grid,
                     scoring = 'neg_mean_absolute_error',
                     n_jobs     =  20,
                     cv         = group_kfold, 
                     refit      = True,
                     verbose    = 1,
                     return_train_score = True
                     )


In [ ]:
grid1.fit(X_train,Y_train, groups)

In [ ]:
TrainValMAE=grid1.cv_results_

In [ ]:
grid1.best_params_

In [ ]:
TrainValMAE

In [ ]:
ErrorValMAE=-1*(TrainValMAE.get('split0_test_score')[17]+TrainValMAE.get('split1_test_score')[17]+TrainValMAE.get('split2_test_score')[17])/3
ErrorValMAE

In [ ]:
ErrorTrainMAE=-1*(TrainValMAE.get('split0_train_score')[17]+TrainValMAE.get('split1_train_score')[17]+TrainValMAE.get('split2_train_score')[17])/3
ErrorTrainMAE

In [ ]:
-1*grid1.best_score_#confirma el calculo del error de validacion

In [ ]:
ModelMAE=grid1.best_estimator_

In [ ]:
pred_test_y2= ModelMAE.predict(X_testt)

In [ ]:
pred_test_y2[pred_test_y2<0]=0
print(f"RMSE= {np.sqrt(metrics.mean_squared_error(Y_test,pred_test_y2))}")
print(f"MAE= {metrics.mean_absolute_error(Y_test,pred_test_y2)}")

In [ ]:
test_pred_df = pd.DataFrame({"fullVisitorId":X_test["fullVisitorId"].values})
test_pred_df["transactionRevenue"] =  (Y_test.values)
test_pred_df["PredictedRevenue"] = (pred_test_y2)
test_pred_df = test_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
print('RMSE')
print(np.sqrt(metrics.mean_squared_error(test_pred_df["transactionRevenue"].values, test_pred_df["PredictedRevenue"].values)))
print('MAE')
print((metrics.mean_absolute_error((test_pred_df["transactionRevenue"].values ),(test_pred_df["PredictedRevenue"].values))))